In [2]:
import os

import numpy as np
import fasttext.util

import utils


embeddings = utils.get_embeddings(["en", "fr"])

en_emb = embeddings["en"]
fr_emb = embeddings["fr"]

In [4]:
data = ["the cat likes oranges"]
target = ["le chat aime les oranges"]

class TranslationDataset():
    def __init__(self, sequence, target, embedding):
        self.sequence = sequence 
        self.target = target 
        self.embedding = embedding
        
        for 

    def __len__(self):
        return (len(self.images))

    def __getitem__(self, idx):
        return({
            "":self.images[idx],
            "label":int(self.labels[idx]),
        })

dataset = TranslationDataset(data, target) 

print(dataset[0])

SyntaxError: invalid syntax (<ipython-input-4-53cb71cf2f71>, line 6)